<a href="https://colab.research.google.com/github/OMEAkin/ML_with_TF_Opencampus_SH/blob/main/week_5/Wk5_Nbk_Sentiment140_OMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis Using the Sentiment140 Dataset

In this notebook, we aim to accomplish the following:
1. Import and analyze the Sentiment140 dataset.
2. Prepare the data for model training
3. Implement a train-validation split to evaluate our model's performance.
4. Train a tokenizer on the training data and tokenize the data
5. Pad the data on a selected length
6. Construct and train a neural network model that includes an embedding layer for text representation.
7. Inspect the output dimensionality of the embedding layer.
8. Train the model to classify the sentiment of tweets and evaluate its performance using the validation set.
9. Tune the hyper parameters for the tokenization and model training.

## Introduction
For this exercise, we'll be using a subsample of the Sentiment140 dataset from Kaggle.
Sentiment140 is a popular dataset for sentiment analysis that contains 1.6 million tweets labeled for sentiment. This dataset is widely used for training machine learning models to differentiate between positive and negative sentiment in text.

You can download the dataset [here](https://github.com/opencampus-sh/course-material/blob/main/machine-learning-with-tensorflow/week-05/sentiment140_small.csv).

Please download the dataset and upload it to your Google Drive.
By uploading the data to your Google Drive and not directly to your Colab environment, it is persistently available. You just need to run the cell the code to mount your Google Drive to your Colab environment.

The dataset contains the following 6 fields:  
`target`: the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)  
`ids`: The id of the tweet ( 2087)  
`date`: the date of the tweet (Sat May 16 23:58:44 UTC 2009)  
`flag`: The query (lyx). If there is no query, then this value is NO_QUERY.  
`user`: the user that tweeted (robotickilldozr)  
`text`: the text of the tweet (Lyx is cool)  

## Google Drive Setup
Before proceeding, ensure that you have uploaded the Sentiment140 dataset to your Google Drive in a specified folder. Then, use the following code to mount your Google Drive and access files using the path '/content/drive/MyDrive/'.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## Importing Data
Next, we will import the dataset and examine it using descriptive statistics to gain initial insights.

In [ ]:
import pandas as pd

# Define column names
column_names = ["target", "ids", "date", "flag", "user", "text"]

# Update this to the path of your Sentiment140 dataset file
file_path = '/content/drive/MyDrive/Colab Notebooks/sentiment140_small.csv'
data = pd.read_csv(file_path, encoding='ISO-8859-1', names=column_names)  # Note: The encoding may vary based on your dataset specifics

data.head()  # View the first few rows of the dataset


,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,2200003313,Tue Jun 16 18:18:13 PDT 2009,NO_QUERY,DEWGetMeTho77,@Nkluvr4eva My poor little dumpling In Holmde...
2,0,2262462245,Sat Jun 20 22:33:33 PDT 2009,NO_QUERY,amikanai,not again tila i love tequila
3,0,2181612958,Mon Jun 15 12:03:52 PDT 2009,NO_QUERY,klayerbear,@Rach_x13 im just ill .. got a banging hedache
4,0,1795702941,Thu May 14 08:13:36 PDT 2009,NO_QUERY,MaRaNdA_x0,Might not get to see her boyfriend for a month...


### Descriptive Statistics

In [ ]:
# Average number of words in each tweet
data['text'].apply(lambda x: len(x.split(' '))).mean()

14.385710089311942

In [ ]:
# Summary statistics for numerical columns
data.describe()

,target,ids
count,160001.000000,1.600010e+05
mean,2.004687,1.999298e+09
std,2.000001,1.935712e+08
min,0.000000,1.467810e+09
25%,0.000000,1.957035e+09
50%,4.000000,2.002246e+09
75%,4.000000,2.177264e+09
max,4.000000,2.329205e+09


In [ ]:
# Class distribution
data['target'].value_counts()  # Replace 'your_label_column' with the actual column name for labels

4    80188
0    79813
Name: target, dtype: int64

## Prepare Labels

In [ ]:
# Labels must start at 0 and increase sequentially by 1 to mark the different classes
data['label'] = []

for item in data:
    data['label'].append(item)

data['label'].value_counts()

# Another option would be to use one-hot encoding to represent the labels, where each class is represented by a vector of 0s and a 1 in the position of the class label
# import tensorflow as tf
# import numpy as np

# num_classes = len((data['target']).unique())
# data['labels'] = tf.one_hot(np(data['target']), depth=num_classes)

TypeError: ignored

## Removing Stop Words

Stop words are common words that generally do not contribute much meaning in a sentence and are typically removed in the preprocessing stage of traditional text analysis. This helps in reducing the size of the dataset and improves the performance of the model by focusing on words that carry more meaning.

In this section, we will use the NLTK library, a widely-used Python library for natural language processing, to remove stop words from our dataset.


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def remove_stop_words(data):
    stop_words = set(stopwords.words('english'))
    return [" ".join([word for word in sentence.split() if word.lower() not in stop_words]) for sentence in data]

# Add new column including the texts without stopwords
data['text_without_stopwords'] = remove_stop_words(data['text'])

# print the first few rows of the text columns
data[['text', 'text_without_stopwords']].head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,text_without_stopwords
0,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,@Nkluvr4eva My poor little dumpling In Holmde...,@Nkluvr4eva poor little dumpling Holmdel vids ...
2,not again tila i love tequila,tila love tequila
3,@Rach_x13 im just ill .. got a banging hedache,@Rach_x13 im ill .. got banging hedache
4,Might not get to see her boyfriend for a month...,Might get see boyfriend month mom wont let com...


In [ ]:
# Average number of words in each tweet without stopwords
data['text_without_stopwords'].apply(lambda x: len(x.split(' '))).mean()

8.118430509809313

## Splitting Data into Training and Test Sets

Before training our model, it is essential to split the dataset into training and test sets. This approach helps in assessing the performance of the model on unseen data, ensuring that our evaluations are realistic and our model is not overfitting to the training data.

We will use a typical split ratio of 80% for training and 20% for testing. You can adjust this ratio based on your dataset size and requirements.


In [ ]:
from sklearn.model_selection import train_test_split

# Assuming 'data' is your dataframe and 'labels' is the column with sentiment labels
X_train, X_test, y_train, y_test = train_test_split(
    data['text'], data['your_labels'], test_size=0.2, random_state=42) # replace 'your_text' and 'your_labels' with the actual column names for text and labels

print(f"Training set size: {len(X_train)}")
print(f"Test set size: {len(X_test)}")


## Tokenization Function
The following function will be responsible for tokenizing our text data. We'll use Keras' Tokenizer class, which allows us to vectorize a text corpus by turning each text into a sequence of integers. The `oov_token` parameter is used to handle out-of-vocabulary words during text conversion.


In [ ]:
# Install TensorFlow library for text processing (if not already installed - for Colab you can skip it)
%pip install tensorflow_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 60.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.5 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: keras
    Found existing installation: keras 2.14.0
    Uninstalling keras-2.14.0:
      Successfully uninstalled keras-2.14.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.14.1
    Uninstalling tensorboard-2.14.1:
      Successfully uninstalled tensorboard-2.14.1
  Attempting uninstall: tensorflow
    Found existing install

In [ ]:
# Import libraries
from tensorflow.keras.preprocessing.text import Tokenizer # Implement tokenizer function

# Define the tokenizer
tokenizer = Tokenizer(oov_token="<OOV>", num_words = 0) # Change the num_words parameter to the desired number of words that your dictionary should contain

# Train the tokenizer
# INCLUDE YOUR CODE HERE

# Tokenize the data
X_train_sequences = # INCLUDE YOUR CODE HERE

# Create a datframe with the original and the tokenized data to check the tokenization for the first rows
pd.DataFrame(zip(X_train, X_train_sequences), columns=['text', 'sequence']).head()

## Padding
To ensure consistent input shape for modeling, we apply padding to our tokenized text. Padding adjusts the sequence length so that all inputs are of the same length, which is necessary for batch processing in neural networks.

In this step, it important to decide on the best maximum length of sequences for padding. This length affects both the model's performance and computational efficiency. If the maximum length is too long, it may lead to increased computational costs and may include lots of padding for shorter sequences. On the other hand, if it's too short, valuable information might be lost.

In this section, we will analyze the distribution of sequence lengths in our dataset and choose an appropriate maximum length. This is often a balance between capturing enough information and maintaining computational efficiency.



In [ ]:
# Plot distribution of sequence lengths
import matplotlib.pyplot as plt

plt.hist([len(sequence) for sequence in X_train_sequences], bins=160)
plt.show()

# print maximum sequence length
print("Longest sequence: ", max([len(sequence) for sequence in X_train_sequences]))

In [ ]:
# Import libraries
import numpy as np
from keras.preprocessing.sequence import pad_sequences

max_len = None # Set the maxlen parameter to the desired maximum length of your sequences
# Note: By setting it to `None`, the maximum length of the sequences will be the length of the longest sequence in the data.
X_train_padded_sequences = pad_sequences(X_train_sequences, padding='post', maxlen=max_len)
X_test_padded_sequences = pad_sequences(X_test_sequences, padding='post', maxlen=max_len)

## Model Construction
Now, we will define our neural network model, incorporating an embedding layer to capture text representation effectively.


In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, GlobalAveragePooling1D

# Define model parameters
vocab_size = XXX  # Replace with your vocabulary size as defined in the tokenization step
max_length = XXX  # Replace with your maximum sequence length as defined in the padding step
embedding_dim = 128  # Size of the embedding vectors


# Define the model architecture
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length))
model.add(GlobalAveragePooling1D()),
model.add(Dense(20, activation='relu')),
model.add(Dense(units=NUMBER_OF_CLASSES, activation='softmax'))

# Display the model's architecture
model.summary()

In [ ]:
# Inspect the embedding layer dimensions after training the model
embedding_layer_weights = model.layers[0].get_weights()[0]
print(f"Shape of embedding layer weights: {embedding_layer_weights.shape}")


Shape of embedding layer weights: (10000, 128)


## Model Fitting

In this section, we will compile and train our model.

In [ ]:
# Compile and train the model
model.compile(optimizer='adam', loss='replace_with_suitable_loss_function', metrics=['accuracy']) # The loss function can either be `sparse_categorical_crossentropy` or `categorical_crossentropy`, depending on how you ave encoded your labels
history = model.fit(your_train_texts, your_train_labels, epochs=30, batch_size=1024, validation_data=(your_test_texts, your_test_labels)) # Replace your_train_texts, your_train_labels, your_test_texts, your_test_labels with the actual variables containing your data

Use the code of above to compile and train your data again but replace definition of `validation_data` with `validation_split=0.2` argument (see below).  
Why is this approach of defining the validation data not recommended?

In [ ]:
# Compile and train the model
# INCLUDE YOUR CODE HERE
history_val_split = # INCLUDE YOUR CODE HERE

In [ ]:
# Plot training & validation accuracy values for both models of above
plt.figure(figsize=(12, 6))

# Plot accuracy for the first model
plt.plot(history.history['accuracy'], 'b-', label='Train accuracy Base Model')
plt.plot(history.history['val_accuracy'], 'b--', label='Validation accuracy Base Model')

# Plot accuracy for the second model
plt.plot(history_val_split.history['accuracy'], 'r-', label='Train accuracy Validation Split Model')
plt.plot(history_val_split.history['val_accuracy'], 'r--', label='Validation accuracy Validation Split Model')

plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper right')
plt.show()

## Hyperparameter Tuning

Now that you have a basic understanding of how to construct and train a text classification model using Keras, you can experiment with the model to improve its performance. One way to do this is by tuning the model's hyperparameters.

Here are a few hyperparameters you can experiment with:

1. **Vocabulary Size**: This is the number of unique words in your text data. A larger vocabulary size means the model can recognize more unique words, but it also increases the dimensionality of the data and can lead to overfitting. Try reducing the vocabulary size to see if it improves the model's performance.

2. **Maximum Sequence Length**: This is the length of the input sequences. If you increase the maximum sequence length, the model will be able to process longer sequences, but it will also take longer to train and may be more prone to overfitting. Try decreasing the maximum sequence length to see if it improves the model's performance.

3. **Embedding Dimensionality**: This is the size of the vectors in which words will be embedded. A higher dimensionality can capture more nuanced relationships between words, but it also increases the computational cost and can lead to overfitting. Try experimenting with different embedding dimensionalities to see what works best.

Remember, the goal of hyperparameter tuning is to find the combination of hyperparameters that gives the best performance on your validation data. Happy tuning!